In [4]:
import numpy as np
from scipy.optimize import minimize

W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

def linear_method(w, d):
    return ((w/d-1)**2)

def ranking_ratio_method(w, d):
    return (w/d)*np.log(w/d)-(w/d)+1

# def objective(W):
#     return (1/2)*sum(
#         d_k*linear_method(w_k, d_k) for w_k, d_k in zip(W, D)
#     )

def objective(W):
    return sum(
        d_k*ranking_ratio_method(w_k, d_k) for w_k, d_k in zip(W, D)
    )

def constraint(W):
    return A.T@W-B

constraints = {"type" : "eq", "fun" : constraint}

result = minimize(objective, 
                  x0=x0, 
                  method="trust-constr", 
                  constraints=constraints)

In [20]:
class MarginCalibration:

    def __init__(self,
                sampling_probabilities,
                calibration_matrix,
                calibration_target,
                calibration_method):
        self.sampling_probabilities = sampling_probabilities
        self.calibration_matrix = calibration_matrix
        self.calibration_target = calibration_target
        self.calibration_method = calibration_method

    def initialize_sampling_weights(self):
        return np.array([1/prob_i for prob_i in self.sampling_probabilities])

    def _linear_method(self, w, d):
        return ((w/d-1)**2)

    def _ranking_ratio_method(self, w, d):
        return (w/d)*np.log(w/d)-(w/d)+1

    def _logit_method(self, w, d, lower_bound, upper_bound):
        pass
        
    def initialize_method(self):
        if self.calibration_method == "linear":
            return self._linear_method
        elif self.calibration_method == "ranking_ratio":
            return self._ranking_ratio_method
        elif self.calibration_method == "logit":
            return self._logit_method
        elif self.calibration_method == "truncated_linear":
            return self._truncated_linear
        else:
            raise ValueError(f"Invalid value : {self.calibration_method}. Must be one of : 'linear', 'ranking_ratio'")

    def objective(self, calibration_weights):
        
        D = self.initialize_sampling_weights()
        
        return sum(
            d_k*self.initialize_method()(w_k, d_k) for w_k, d_k in zip(calibration_weights, D)
        )

    def constraint(self, calibration_weights):
        return self.calibration_matrix.T@calibration_weights-self.calibration_target

    def calibration(self):
        
        constraints = {"type":"eq", "fun":self.constraint}
        
        x0 = self.initialize_sampling_weights()
        
        return minimize(
            self.objective,
            x0=x0,
            method = "trust-constr",
            constraints = constraints
        )    

In [21]:
W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

MarginCalibration(W_i, A, B, "linear").calibration()

/opt/conda/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: 228.17016018123127
                x: [ 7.203e+01  1.009e+02  1.638e+02  2.340e+02]
              nit: 20
             nfev: 165
             njev: 33
             nhev: 0
         cg_niter: 21
     cg_stop_cond: 4
             grad: [-5.593e-01  1.782e-02  1.276e+00  2.681e+00]
  lagrangian_grad: [ 1.110e-09  9.172e-08  4.554e-08 -2.205e-08]
           constr: [array([ 0.000e+00,  0.000e+00])]
              jac: [array([[ 1.000e+03,  0.000e+00,  2.500e+01,  1.020e+02],
                          [ 2.584e+03,  4.000e+00,  3.540e+02,  8.780e+02]])]
      constr_nfev: [165]
      constr_njev: [0]
      constr_nhev: [0]
                v: [array([ 1.207e-02, -4.456e-03])]
           method: equality_constrained_sqp
       optimality: 9.171765247284469e-08
 constr_violation: 0.0
   execution_time: 0.03226828575134277
        tr_radius: 9.063784488986924e-09
   constr